# Métodos Basados en Modelos y Otros Algoritmos


## GMM, Clustering Espectral, Mean-Shift, Affinity Propagation y BIRCH

### Objetivos del Notebook

1. Implementar Gaussian Mixture Models y comprender el algoritmo EM
2. Aplicar criterios de selección de modelos (BIC, AIC) para determinar el número de componentes
3. Dominar el clustering espectral para estructuras no convexas complejas
4. Explorar algoritmos complementarios: Mean-Shift, Affinity Propagation y BIRCH
5. Comparar el rendimiento de diferentes métodos en escenarios realistas

---

## 1. Configuración del Entorno

Importamos las bibliotecas necesarias para el desarrollo del módulo.

In [ ]:
# Bibliotecas fundamentales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn: clustering
from sklearn.mixture import GaussianMixture
from sklearn.cluster import (
    SpectralClustering, MeanShift, AffinityPropagation, Birch, KMeans
)
from sklearn.cluster import estimate_bandwidth

# Scikit-learn: datasets y métricas
from sklearn.datasets import (
    make_blobs, make_moons, make_circles, load_iris, load_wine
)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    silhouette_score, adjusted_rand_score, normalized_mutual_info_score
)

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['axes.labelsize'] = 11

# Reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Entorno configurado correctamente.")

## 2. Gaussian Mixture Models (GMM)

### 2.1 Concepto y Modelo Probabilístico

Un GMM modela los datos como una mezcla de K distribuciones gaussianas:

$$p(\mathbf{x}) = \sum_{k=1}^{K} \pi_k \cdot \mathcal{N}(\mathbf{x} | \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$$

donde $\pi_k$ son los pesos de mezcla (probabilidades a priori) y cada componente tiene su propia media $\boldsymbol{\mu}_k$ y covarianza $\boldsymbol{\Sigma}_k$.

In [ ]:
# Generar datos con clusters elípticos de diferente orientación
np.random.seed(RANDOM_STATE)

# Cluster 1: elipse orientada a 45 grados
n1 = 300
theta1 = np.pi / 4
cov1 = np.array([[2, 0], [0, 0.3]])
rotation1 = np.array([[np.cos(theta1), -np.sin(theta1)],
                      [np.sin(theta1), np.cos(theta1)]])
cov1_rotated = rotation1 @ cov1 @ rotation1.T
cluster1 = np.random.multivariate_normal([0, 0], cov1_rotated, n1)

# Cluster 2: elipse orientada a -30 grados
n2 = 250
theta2 = -np.pi / 6
cov2 = np.array([[1.5, 0], [0, 0.4]])
rotation2 = np.array([[np.cos(theta2), -np.sin(theta2)],
                      [np.sin(theta2), np.cos(theta2)]])
cov2_rotated = rotation2 @ cov2 @ rotation2.T
cluster2 = np.random.multivariate_normal([4, 3], cov2_rotated, n2)

# Cluster 3: circular
n3 = 100
cluster3 = np.random.multivariate_normal([2, -2], [[0.5, 0], [0, 0.5]], n3)

# Combinar datos
X_ellipse = np.vstack([cluster1, cluster2, cluster3])
y_ellipse = np.array([0]*n1 + [1]*n2 + [2]*n3)

print(f"Dataset generado: {len(X_ellipse)} puntos en 3 clusters elípticos")

In [ ]:
# Comparar K-Means vs GMM en datos elípticos
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Ground truth
ax = axes[0]
ax.scatter(X_ellipse[:, 0], X_ellipse[:, 1], c=y_ellipse, cmap='viridis',
           edgecolors='w', s=40, alpha=0.7)
ax.set_title('Ground Truth')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

# K-Means
ax = axes[1]
kmeans = KMeans(n_clusters=3, random_state=RANDOM_STATE, n_init=10)
labels_km = kmeans.fit_predict(X_ellipse)
ari_km = adjusted_rand_score(y_ellipse, labels_km)
ax.scatter(X_ellipse[:, 0], X_ellipse[:, 1], c=labels_km, cmap='viridis',
           edgecolors='w', s=40, alpha=0.7)
ax.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
           c='red', marker='X', s=200, edgecolors='w', linewidths=2)
ax.set_title(f'K-Means (ARI: {ari_km:.3f})')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

# GMM
ax = axes[2]
gmm = GaussianMixture(n_components=3, covariance_type='full', random_state=RANDOM_STATE)
labels_gmm = gmm.fit_predict(X_ellipse)
ari_gmm = adjusted_rand_score(y_ellipse, labels_gmm)
ax.scatter(X_ellipse[:, 0], X_ellipse[:, 1], c=labels_gmm, cmap='viridis',
           edgecolors='w', s=40, alpha=0.7)
ax.scatter(gmm.means_[:, 0], gmm.means_[:, 1],
           c='red', marker='X', s=200, edgecolors='w', linewidths=2)
ax.set_title(f'GMM (ARI: {ari_gmm:.3f})')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

plt.suptitle('Comparación K-Means vs GMM en clusters elípticos', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"\nK-Means asume clusters esféricos y falla con elipses orientadas.")
print(f"GMM captura la forma elíptica de cada cluster.")

### 2.2 Tipos de Covarianza

GMM permite diferentes restricciones en las matrices de covarianza, afectando la flexibilidad y el número de parámetros del modelo.

In [ ]:
def plot_gmm_ellipses(gmm, ax, colors):
    """
    Dibuja las elipses de covarianza de un GMM.
    """
    for i, (mean, covar, color) in enumerate(zip(gmm.means_, gmm.covariances_, colors)):
        if gmm.covariance_type == 'full':
            cov = covar
        elif gmm.covariance_type == 'tied':
            cov = gmm.covariances_
        elif gmm.covariance_type == 'diag':
            cov = np.diag(covar)
        elif gmm.covariance_type == 'spherical':
            cov = np.eye(2) * covar

        # Calcular eigenvalores y eigenvectores
        eigenvalues, eigenvectors = np.linalg.eigh(cov)
        order = eigenvalues.argsort()[::-1]
        eigenvalues = eigenvalues[order]
        eigenvectors = eigenvectors[:, order]

        # Ángulo de rotación
        angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))

        # Dibujar elipses para 1, 2 y 3 desviaciones estándar
        for n_std in [1, 2]:
            width = 2 * n_std * np.sqrt(eigenvalues[0])
            height = 2 * n_std * np.sqrt(eigenvalues[1])
            ellipse = Ellipse(mean, width, height, angle=angle,
                            fill=False, edgecolor=color, linewidth=2,
                            alpha=0.8 if n_std == 1 else 0.4)
            ax.add_patch(ellipse)

In [ ]:
# Comparar tipos de covarianza
cov_types = ['full', 'tied', 'diag', 'spherical']
colors = ['#e74c3c', '#3498db', '#2ecc71']

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

for ax, cov_type in zip(axes.flatten(), cov_types):
    gmm = GaussianMixture(n_components=3, covariance_type=cov_type,
                          random_state=RANDOM_STATE, n_init=5)
    labels = gmm.fit_predict(X_ellipse)
    ari = adjusted_rand_score(y_ellipse, labels)
    bic = gmm.bic(X_ellipse)

    ax.scatter(X_ellipse[:, 0], X_ellipse[:, 1], c=labels, cmap='viridis',
               edgecolors='w', s=30, alpha=0.6)
    plot_gmm_ellipses(gmm, ax, colors)
    ax.scatter(gmm.means_[:, 0], gmm.means_[:, 1], c='red', marker='X',
               s=150, edgecolors='w', linewidths=2, zorder=5)

    ax.set_title(f'{cov_type.capitalize()}\nARI: {ari:.3f}, BIC: {bic:.0f}')
    ax.set_xlabel('Característica 1')
    ax.set_ylabel('Característica 2')
    ax.set_xlim(-4, 8)
    ax.set_ylim(-5, 7)

plt.suptitle('Tipos de Covarianza en GMM', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("Interpretación de las elipses:")
print("- Full: cada componente tiene su propia matriz de covarianza completa")
print("- Tied: todas las componentes comparten la misma covarianza")
print("- Diagonal: solo varianzas por eje (elipses alineadas con ejes)")
print("- Spherical: varianza única por componente (círculos)")

### 2.3 Selección del Número de Componentes con BIC y AIC

In [ ]:
# Generar dataset más complejo para selección de modelo
np.random.seed(RANDOM_STATE)

# 5 clusters con diferentes características
X_complex, y_complex = make_blobs(n_samples=600, centers=5,
                                   cluster_std=[0.8, 1.2, 0.6, 1.0, 0.9],
                                   random_state=RANDOM_STATE)

# Calcular BIC y AIC para diferentes números de componentes
n_components_range = range(1, 12)
bics = []
aics = []

for n in n_components_range:
    gmm = GaussianMixture(n_components=n, covariance_type='full',
                          random_state=RANDOM_STATE, n_init=5)
    gmm.fit(X_complex)
    bics.append(gmm.bic(X_complex))
    aics.append(gmm.aic(X_complex))

# Visualización
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# BIC y AIC
ax = axes[0]
ax.plot(n_components_range, bics, 'b-o', label='BIC', linewidth=2, markersize=8)
ax.plot(n_components_range, aics, 'r--s', label='AIC', linewidth=2, markersize=8)
ax.axvline(x=np.argmin(bics)+1, color='blue', linestyle=':', alpha=0.7, label=f'BIC óptimo: {np.argmin(bics)+1}')
ax.axvline(x=np.argmin(aics)+1, color='red', linestyle=':', alpha=0.7, label=f'AIC óptimo: {np.argmin(aics)+1}')
ax.set_xlabel('Número de componentes')
ax.set_ylabel('Valor del criterio')
ax.set_title('Selección de K con BIC y AIC')
ax.legend()
ax.grid(True, alpha=0.3)

# Ground truth
ax = axes[1]
ax.scatter(X_complex[:, 0], X_complex[:, 1], c=y_complex, cmap='viridis',
           edgecolors='w', s=40)
ax.set_title(f'Ground Truth (K=5)')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

# GMM con K óptimo según BIC
ax = axes[2]
k_optimo = np.argmin(bics) + 1
k_optimo = 4
gmm_optimo = GaussianMixture(n_components=k_optimo, covariance_type='full',
                              random_state=RANDOM_STATE, n_init=5)
labels_optimo = gmm_optimo.fit_predict(X_complex)
ari_optimo = adjusted_rand_score(y_complex, labels_optimo)
ax.scatter(X_complex[:, 0], X_complex[:, 1], c=labels_optimo, cmap='viridis',
           edgecolors='w', s=40)
ax.set_title(f'GMM con K={k_optimo} (BIC óptimo)\nARI: {ari_optimo:.3f}')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

plt.tight_layout()
plt.show()

print(f"\nK óptimo según BIC: {np.argmin(bics)+1}")
print(f"K óptimo según AIC: {np.argmin(aics)+1}")
print(f"K real: 5")

### 2.4 Soft Clustering: Probabilidades de Pertenencia

In [ ]:
# Entrenar GMM y obtener probabilidades
gmm_soft = GaussianMixture(n_components=3, covariance_type='full',
                           random_state=RANDOM_STATE)
gmm_soft.fit(X_ellipse)

# Probabilidades de pertenencia (soft clustering)
probs = gmm_soft.predict_proba(X_ellipse)

# Etiquetas (hard clustering)
labels_hard = gmm_soft.predict(X_ellipse)

# Calcular incertidumbre (entropía normalizada)
def entropy(p):
    """Calcula la entropía de una distribución de probabilidad."""
    p = np.clip(p, 1e-10, 1)  # Evitar log(0)
    return -np.sum(p * np.log(p), axis=1)

incertidumbre = entropy(probs) / np.log(3)  # Normalizado [0, 1]

# Visualización
fig, axes = plt.subplots(2, 3, figsize=(14, 12))

# Hard clustering
ax = axes[0, 0]
ax.scatter(X_ellipse[:, 0], X_ellipse[:, 1], c=labels_hard, cmap='viridis',
           edgecolors='w', s=40)
ax.set_title('Hard Clustering (etiqueta más probable)')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

# Incertidumbre
ax = axes[0, 1]
scatter = ax.scatter(X_ellipse[:, 0], X_ellipse[:, 1], c=incertidumbre,
                     cmap='Reds', edgecolors='w', s=40)
plt.colorbar(scatter, ax=ax, label='Incertidumbre (entropía normalizada)')
ax.set_title('Incertidumbre en la asignación')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

# Probabilidad para cada componente
for i in range(3):
    if i < 2:
        ax = axes[1, i]
        scatter = ax.scatter(X_ellipse[:, 0], X_ellipse[:, 1], c=probs[:, i],
                             cmap='Blues', edgecolors='w', s=40, vmin=0, vmax=1)
        plt.colorbar(scatter, ax=ax, label=f'P(componente {i})')
        ax.set_title(f'Probabilidad de pertenencia al componente {i}')
        ax.set_xlabel('Característica 1')
        ax.set_ylabel('Característica 2')

plt.suptitle('Soft Clustering con GMM', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Estadísticas de incertidumbre
print(f"\nEstadísticas de incertidumbre:")
print(f"  Media: {incertidumbre.mean():.3f}")
print(f"  Máxima: {incertidumbre.max():.3f}")
print(f"  Puntos con alta incertidumbre (>0.5): {(incertidumbre > 0.5).sum()}")

In [ ]:
# Ejemplo de puntos con diferentes niveles de certeza
print("Ejemplos de probabilidades de pertenencia:\n")

# Punto con alta certeza
idx_alta_certeza = np.argmin(incertidumbre)
print(f"Punto con ALTA certeza (índice {idx_alta_certeza}):")
print(f"  Probabilidades: {probs[idx_alta_certeza]}")
print(f"  Incertidumbre: {incertidumbre[idx_alta_certeza]:.4f}")

# Punto con baja certeza
idx_baja_certeza = np.argmax(incertidumbre)
print(f"\nPunto con BAJA certeza (índice {idx_baja_certeza}):")
print(f"  Probabilidades: {probs[idx_baja_certeza]}")
print(f"  Incertidumbre: {incertidumbre[idx_baja_certeza]:.4f}")

## 3. Clustering Espectral

### 3.1 Fundamentos: Datos No Convexos Complejos

In [ ]:
# Crear dataset con estructuras no convexas complejas
np.random.seed(RANDOM_STATE)

# Anillos concéntricos con ruido
n_samples = 400
t = np.linspace(0, 2*np.pi, n_samples // 2)

# Anillo exterior
r1 = 2 + 0.1 * np.random.randn(n_samples // 2)
x1 = r1 * np.cos(t) + 0.1 * np.random.randn(n_samples // 2)
y1 = r1 * np.sin(t) + 0.1 * np.random.randn(n_samples // 2)

# Anillo interior
r2 = 1 + 0.1 * np.random.randn(n_samples // 2)
x2 = r2 * np.cos(t) + 0.1 * np.random.randn(n_samples // 2)
y2 = r2 * np.sin(t) + 0.1 * np.random.randn(n_samples // 2)

X_rings = np.vstack([np.column_stack([x1, y1]), np.column_stack([x2, y2])])
y_rings = np.array([0] * (n_samples // 2) + [1] * (n_samples // 2))

# Espirales entrelazadas
n_spiral = 300
t_spiral = np.linspace(0, 3*np.pi, n_spiral)
noise = 0.3

# Espiral 1
x1_spiral = t_spiral * np.cos(t_spiral) + noise * np.random.randn(n_spiral)
y1_spiral = t_spiral * np.sin(t_spiral) + noise * np.random.randn(n_spiral)

# Espiral 2 (rotada 180 grados)
x2_spiral = t_spiral * np.cos(t_spiral + np.pi) + noise * np.random.randn(n_spiral)
y2_spiral = t_spiral * np.sin(t_spiral + np.pi) + noise * np.random.randn(n_spiral)

X_spirals = np.vstack([np.column_stack([x1_spiral, y1_spiral]),
                       np.column_stack([x2_spiral, y2_spiral])])
y_spirals = np.array([0] * n_spiral + [1] * n_spiral)

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

ax = axes[0]
ax.scatter(X_rings[:, 0], X_rings[:, 1], c=y_rings, cmap='viridis',
           edgecolors='w', s=30)
ax.set_title('Anillos concéntricos')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_aspect('equal')

ax = axes[1]
ax.scatter(X_spirals[:, 0], X_spirals[:, 1], c=y_spirals, cmap='viridis',
           edgecolors='w', s=30)
ax.set_title('Espirales entrelazadas')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_aspect('equal')

plt.suptitle('Datasets con estructuras no convexas', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Comparar K-Means, GMM y Espectral en anillos
fig, axes = plt.subplots(2, 4, figsize=(18, 9))

datasets = [
    (X_rings, y_rings, 'Anillos'),
    (X_spirals, y_spirals, 'Espirales')
]

for row, (X, y_true, titulo) in enumerate(datasets):
    # Ground truth
    ax = axes[row, 0]
    ax.scatter(X[:, 0], X[:, 1], c=y_true, cmap='viridis', edgecolors='w', s=25)
    ax.set_title('Ground Truth')
    if row == 0:
        ax.set_ylabel(titulo, fontsize=12, fontweight='bold')
    ax.set_aspect('equal')

    # K-Means
    ax = axes[row, 1]
    kmeans = KMeans(n_clusters=2, random_state=RANDOM_STATE, n_init=10)
    labels_km = kmeans.fit_predict(X)
    ari_km = adjusted_rand_score(y_true, labels_km)
    ax.scatter(X[:, 0], X[:, 1], c=labels_km, cmap='viridis', edgecolors='w', s=25)
    ax.set_title(f'K-Means\nARI: {ari_km:.3f}')
    ax.set_aspect('equal')

    # GMM
    ax = axes[row, 2]
    gmm = GaussianMixture(n_components=2, covariance_type='full', random_state=RANDOM_STATE)
    labels_gmm = gmm.fit_predict(X)
    ari_gmm = adjusted_rand_score(y_true, labels_gmm)
    ax.scatter(X[:, 0], X[:, 1], c=labels_gmm, cmap='viridis', edgecolors='w', s=25)
    ax.set_title(f'GMM\nARI: {ari_gmm:.3f}')
    ax.set_aspect('equal')

    # Espectral
    ax = axes[row, 3]
    spectral = SpectralClustering(n_clusters=2, affinity='nearest_neighbors',
                                   n_neighbors=10, random_state=RANDOM_STATE)
    labels_spec = spectral.fit_predict(X)
    ari_spec = adjusted_rand_score(y_true, labels_spec)
    ax.scatter(X[:, 0], X[:, 1], c=labels_spec, cmap='viridis', edgecolors='w', s=25)
    ax.set_title(f'Espectral\nARI: {ari_spec:.3f}')
    ax.set_aspect('equal')

plt.suptitle('Comparación de algoritmos en estructuras no convexas',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### 3.2 Efecto del Parámetro de Afinidad

In [ ]:
# Comparar diferentes configuraciones de clustering espectral
fig, axes = plt.subplots(2, 4, figsize=(18, 9))

# Configuraciones a probar
configs = [
    {'affinity': 'rbf', 'gamma': 0.1, 'label': 'RBF (gamma=0.1)'},
    {'affinity': 'rbf', 'gamma': 1.0, 'label': 'RBF (gamma=1.0)'},
    {'affinity': 'rbf', 'gamma': 10.0, 'label': 'RBF (gamma=10)'},
    {'affinity': 'nearest_neighbors', 'n_neighbors': 10, 'label': 'k-NN (k=10)'}
]

for row, (X, y_true, titulo) in enumerate(datasets):
    for col, config in enumerate(configs):
        ax = axes[row, col]

        # Configurar clustering espectral
        if config['affinity'] == 'rbf':
            spectral = SpectralClustering(n_clusters=2, affinity='rbf',
                                          gamma=config['gamma'],
                                          random_state=RANDOM_STATE)
        else:
            spectral = SpectralClustering(n_clusters=2, affinity='nearest_neighbors',
                                          n_neighbors=config['n_neighbors'],
                                          random_state=RANDOM_STATE)

        labels = spectral.fit_predict(X)
        ari = adjusted_rand_score(y_true, labels)

        ax.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', edgecolors='w', s=25)
        ax.set_title(f"{config['label']}\nARI: {ari:.3f}")
        ax.set_aspect('equal')

        if col == 0:
            ax.set_ylabel(titulo, fontsize=12, fontweight='bold')

plt.suptitle('Efecto de los parámetros en Clustering Espectral',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("Observaciones:")
print("- gamma bajo (RBF): considera puntos más lejanos como similares")
print("- gamma alto (RBF): solo puntos muy cercanos son similares")
print("- k-NN: conectividad local basada en vecinos más cercanos")

## 4. Otros Algoritmos de Clustering

### 4.1 Mean-Shift

In [ ]:
# Generar datos para Mean-Shift (detecta número de clusters automáticamente)
np.random.seed(RANDOM_STATE)

# Clusters con diferentes densidades
X_ms, y_ms = make_blobs(n_samples=500, centers=4,
                        cluster_std=[0.5, 1.0, 0.7, 0.8],
                        random_state=RANDOM_STATE)

# Estimar bandwidth automáticamente
bandwidth = estimate_bandwidth(X_ms, quantile=0.2)
print(f"Bandwidth estimado: {bandwidth:.3f}")

# Aplicar Mean-Shift
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
labels_ms = ms.fit_predict(X_ms)
n_clusters_ms = len(np.unique(labels_ms))

# Visualización
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Ground truth
ax = axes[0]
ax.scatter(X_ms[:, 0], X_ms[:, 1], c=y_ms, cmap='viridis', edgecolors='w', s=40)
ax.set_title('Ground Truth (K=4)')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

# Mean-Shift
ax = axes[1]
ax.scatter(X_ms[:, 0], X_ms[:, 1], c=labels_ms, cmap='viridis', edgecolors='w', s=40)
ax.scatter(ms.cluster_centers_[:, 0], ms.cluster_centers_[:, 1],
           c='red', marker='X', s=200, edgecolors='w', linewidths=2)
ari_ms = adjusted_rand_score(y_ms, labels_ms)
ax.set_title(f'Mean-Shift (K detectado: {n_clusters_ms})\nARI: {ari_ms:.3f}')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

# Efecto del bandwidth
ax = axes[2]
bandwidths = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
n_clusters_list = []
aris = []

for bw in bandwidths:
    ms_test = MeanShift(bandwidth=bw, bin_seeding=True)
    labels_test = ms_test.fit_predict(X_ms)
    n_clusters_list.append(len(np.unique(labels_test)))
    aris.append(adjusted_rand_score(y_ms, labels_test))

ax.plot(bandwidths, n_clusters_list, 'b-o', linewidth=2, markersize=8, label='Clusters')
ax.axhline(y=4, color='gray', linestyle='--', alpha=0.7, label='K real')
ax.axvline(x=bandwidth, color='red', linestyle=':', alpha=0.7, label=f'BW estimado: {bandwidth:.2f}')
ax.set_xlabel('Bandwidth')
ax.set_ylabel('Número de clusters')
ax.set_title('Efecto del bandwidth')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 4.2 Affinity Propagation

In [ ]:
# Dataset pequeño para Affinity Propagation (computacionalmente costoso)
X_ap, y_ap = make_blobs(n_samples=200, centers=5, cluster_std=0.8,
                        random_state=RANDOM_STATE)

# Probar diferentes valores de preference
preferences = [None, -50, -100, -200]

fig, axes = plt.subplots(1, 4, figsize=(18, 4))

for ax, pref in zip(axes, preferences):
    ap = AffinityPropagation(damping=0.9, preference=pref, random_state=RANDOM_STATE)
    labels_ap = ap.fit_predict(X_ap)
    n_clusters_ap = len(np.unique(labels_ap))
    ari_ap = adjusted_rand_score(y_ap, labels_ap)

    # Ejemplares (puntos representativos)
    ejemplares = ap.cluster_centers_indices_

    ax.scatter(X_ap[:, 0], X_ap[:, 1], c=labels_ap, cmap='viridis',
               edgecolors='w', s=40)
    ax.scatter(X_ap[ejemplares, 0], X_ap[ejemplares, 1],
               c='red', marker='X', s=200, edgecolors='w', linewidths=2)

    pref_str = 'Auto' if pref is None else str(pref)
    ax.set_title(f'Preference: {pref_str}\nK: {n_clusters_ap}, ARI: {ari_ap:.3f}')
    ax.set_xlabel('Característica 1')
    ax.set_ylabel('Característica 2')

plt.suptitle('Affinity Propagation: Efecto del parámetro preference',
             fontsize=14, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

print("Nota: Los ejemplares (X rojas) son puntos reales del dataset, no centroides calculados.")
print("Preference más negativo = menos clusters.")

### 4.3 BIRCH para Grandes Volúmenes de Datos

In [ ]:
# Generar dataset grande
np.random.seed(RANDOM_STATE)
n_large = 10000

X_large, y_large = make_blobs(n_samples=n_large, centers=6,
                               cluster_std=1.0, random_state=RANDOM_STATE)

print(f"Dataset generado: {n_large} puntos, 6 clusters")

# Comparar tiempos de ejecución
import time

resultados = []

# BIRCH
start = time.time()
birch = Birch(n_clusters=6, threshold=0.5, branching_factor=50)
labels_birch = birch.fit_predict(X_large)
tiempo_birch = time.time() - start
ari_birch = adjusted_rand_score(y_large, labels_birch)
resultados.append(('BIRCH', tiempo_birch, ari_birch))

# K-Means
start = time.time()
kmeans_large = KMeans(n_clusters=6, random_state=RANDOM_STATE, n_init=10)
labels_kmeans_large = kmeans_large.fit_predict(X_large)
tiempo_kmeans = time.time() - start
ari_kmeans = adjusted_rand_score(y_large, labels_kmeans_large)
resultados.append(('K-Means', tiempo_kmeans, ari_kmeans))

# GMM
start = time.time()
gmm_large = GaussianMixture(n_components=6, random_state=RANDOM_STATE, n_init=3)
labels_gmm_large = gmm_large.fit_predict(X_large)
tiempo_gmm = time.time() - start
ari_gmm = adjusted_rand_score(y_large, labels_gmm_large)
resultados.append(('GMM', tiempo_gmm, ari_gmm))

# Mostrar resultados
print("\nComparación de rendimiento (n=10,000):")
print("=" * 45)
print(f"{'Algoritmo':<15} {'Tiempo (s)':<15} {'ARI':<10}")
print("-" * 45)
for nombre, tiempo, ari in resultados:
    print(f"{nombre:<15} {tiempo:<15.4f} {ari:<10.4f}")

In [ ]:
# Visualización del clustering en dataset grande
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

algoritmos = [
    ('BIRCH', labels_birch, ari_birch, tiempo_birch),
    ('K-Means', labels_kmeans_large, ari_kmeans, tiempo_kmeans),
    ('GMM', labels_gmm_large, ari_gmm, tiempo_gmm)
]

for ax, (nombre, labels, ari, tiempo) in zip(axes, algoritmos):
    # Submuestrear para visualización
    idx = np.random.choice(len(X_large), 2000, replace=False)
    ax.scatter(X_large[idx, 0], X_large[idx, 1], c=labels[idx],
               cmap='viridis', s=10, alpha=0.6)
    ax.set_title(f'{nombre}\nARI: {ari:.3f}, Tiempo: {tiempo:.3f}s')
    ax.set_xlabel('Característica 1')
    ax.set_ylabel('Característica 2')

plt.suptitle(f'Comparación en dataset grande (n={n_large})',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Escalabilidad de BIRCH vs otros algoritmos
tamanios = [500, 1000, 2000, 5000, 10000, 20000]
tiempos_birch = []
tiempos_kmeans = []

for n in tamanios:
    X_test, _ = make_blobs(n_samples=n, centers=6, random_state=RANDOM_STATE)

    # BIRCH
    start = time.time()
    Birch(n_clusters=6).fit_predict(X_test)
    tiempos_birch.append(time.time() - start)

    # K-Means
    start = time.time()
    KMeans(n_clusters=6, n_init=3, random_state=RANDOM_STATE).fit_predict(X_test)
    tiempos_kmeans.append(time.time() - start)

# Visualización de escalabilidad
plt.figure(figsize=(10, 6))
plt.plot(tamanios, tiempos_birch, 'b-o', linewidth=2, markersize=8, label='BIRCH')
plt.plot(tamanios, tiempos_kmeans, 'r-s', linewidth=2, markersize=8, label='K-Means')
plt.xlabel('Número de muestras')
plt.ylabel('Tiempo (segundos)')
plt.title('Escalabilidad: BIRCH vs K-Means')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("BIRCH escala linealmente O(n), ideal para datasets muy grandes.")

## 5. Caso Práctico: Dataset Wine

Aplicamos los diferentes algoritmos a un dataset real multidimensional.

In [ ]:
# Cargar dataset Wine
wine = load_wine()
X_wine = wine.data
y_wine = wine.target

print(f"Dataset Wine:")
print(f"  Muestras: {X_wine.shape[0]}")
print(f"  Características: {X_wine.shape[1]}")
print(f"  Clases: {len(np.unique(y_wine))} ({np.unique(y_wine)})")
print(f"\nCaracterísticas: {wine.feature_names}")

# Estandarizar
scaler = StandardScaler()
X_wine_scaled = scaler.fit_transform(X_wine)

# Reducir a 2D para visualización
pca = PCA(n_components=2)
X_wine_2d = pca.fit_transform(X_wine_scaled)
print(f"\nVarianza explicada por 2 componentes: {pca.explained_variance_ratio_.sum():.2%}")

In [ ]:
# Selección de K para GMM usando BIC
n_components_range = range(1, 10)
bics_wine = []

for n in n_components_range:
    gmm = GaussianMixture(n_components=n, covariance_type='full',
                          random_state=RANDOM_STATE, n_init=5)
    gmm.fit(X_wine_scaled)
    bics_wine.append(gmm.bic(X_wine_scaled))

plt.figure(figsize=(10, 5))
plt.plot(n_components_range, bics_wine, 'b-o', linewidth=2, markersize=8)
plt.axvline(x=np.argmin(bics_wine)+1, color='red', linestyle='--',
            label=f'K óptimo: {np.argmin(bics_wine)+1}')
plt.axvline(x=3, color='green', linestyle=':', alpha=0.7, label='K real: 3')
plt.xlabel('Número de componentes')
plt.ylabel('BIC')
plt.title('Selección de K con BIC para dataset Wine')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Comparación completa de algoritmos en Wine
fig, axes = plt.subplots(2, 4, figsize=(18, 9))
k = 3

# Definir algoritmos
algoritmos = [
    ('Ground Truth', None, y_wine),
    ('K-Means', KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10), None),
    ('GMM', GaussianMixture(n_components=k, covariance_type='full', random_state=RANDOM_STATE), None),
    ('Espectral', SpectralClustering(n_clusters=k, affinity='rbf', gamma=0.1, random_state=RANDOM_STATE), None),
    ('Mean-Shift', MeanShift(bandwidth=estimate_bandwidth(X_wine_scaled, quantile=0.3)), None),
    ('Affinity Prop.', AffinityPropagation(damping=0.9, preference=-50, random_state=RANDOM_STATE), None),
    ('BIRCH', Birch(n_clusters=k, threshold=1.0), None),
]

resultados_wine = []

for idx, (nombre, modelo, labels_predef) in enumerate(algoritmos):
    row = idx // 4
    col = idx % 4
    ax = axes[row, col]

    if labels_predef is not None:
        labels = labels_predef
        ari = 1.0
        nmi = 1.0
    else:
        labels = modelo.fit_predict(X_wine_scaled)
        ari = adjusted_rand_score(y_wine, labels)
        nmi = normalized_mutual_info_score(y_wine, labels)
        resultados_wine.append((nombre, ari, nmi, len(np.unique(labels))))

    ax.scatter(X_wine_2d[:, 0], X_wine_2d[:, 1], c=labels, cmap='viridis',
               edgecolors='w', s=40, alpha=0.7)

    if nombre == 'Ground Truth':
        ax.set_title(f'{nombre}')
    else:
        n_clusters = len(np.unique(labels))
        ax.set_title(f'{nombre}\nK={n_clusters}, ARI={ari:.3f}')

    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')

# Ocultar el último subplot
axes[1, 3].axis('off')

plt.suptitle('Comparación de algoritmos en dataset Wine',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Tabla comparativa de resultados
print("\nResultados comparativos en dataset Wine:")
print("=" * 55)
print(f"{'Algoritmo':<18} {'ARI':<10} {'NMI':<10} {'K detectado':<12}")
print("-" * 55)
for nombre, ari, nmi, k in resultados_wine:
    print(f"{nombre:<18} {ari:<10.4f} {nmi:<10.4f} {k:<12}")
print("-" * 55)
print(f"{'(K real = 3)':<18}")

## 6. Caso Práctico Avanzado: Segmentación de Imágenes

Aplicamos GMM para segmentar una imagen basándose en sus colores.

In [ ]:
# Generar imagen sintética con regiones de colores
np.random.seed(RANDOM_STATE)

# Crear imagen 100x100 con 4 regiones de colores
img_size = 100
img = np.zeros((img_size, img_size, 3))

# Región 1: Rojo (esquina superior izquierda)
img[:50, :50] = [0.8, 0.2, 0.1] + np.random.randn(50, 50, 3) * 0.05

# Región 2: Verde (esquina superior derecha)
img[:50, 50:] = [0.2, 0.7, 0.2] + np.random.randn(50, 50, 3) * 0.05

# Región 3: Azul (esquina inferior izquierda)
img[50:, :50] = [0.1, 0.3, 0.8] + np.random.randn(50, 50, 3) * 0.05

# Región 4: Amarillo (esquina inferior derecha)
img[50:, 50:] = [0.9, 0.8, 0.2] + np.random.randn(50, 50, 3) * 0.05

# Clipear valores
img = np.clip(img, 0, 1)

# Añadir un círculo de otro color en el centro
center = (50, 50)
radius = 20
for i in range(img_size):
    for j in range(img_size):
        if (i - center[0])**2 + (j - center[1])**2 < radius**2:
            img[i, j] = [0.5, 0.5, 0.5] + np.random.randn(3) * 0.03

img = np.clip(img, 0, 1)

plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.title('Imagen original')
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Preparar datos para clustering
# Cada pixel es un punto en espacio RGB 3D
X_img = img.reshape(-1, 3)

print(f"Datos de imagen: {X_img.shape[0]} píxeles, {X_img.shape[1]} canales (RGB)")

# Selección de K con BIC
n_components_range = range(2, 10)
bics_img = []

for n in n_components_range:
    gmm = GaussianMixture(n_components=n, covariance_type='full',
                          random_state=RANDOM_STATE, n_init=3)
    gmm.fit(X_img)
    bics_img.append(gmm.bic(X_img))

k_optimo_img = n_components_range[np.argmin(bics_img)]

plt.figure(figsize=(10, 5))
plt.plot(list(n_components_range), bics_img, 'b-o', linewidth=2, markersize=8)
plt.axvline(x=k_optimo_img, color='red', linestyle='--',
            label=f'K óptimo: {k_optimo_img}')
plt.xlabel('Número de componentes')
plt.ylabel('BIC')
plt.title('Selección de K para segmentación de imagen')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Segmentación con GMM
gmm_img = GaussianMixture(n_components=k_optimo_img, covariance_type='full',
                          random_state=RANDOM_STATE, n_init=5)
labels_img = gmm_img.fit_predict(X_img)
probs_img = gmm_img.predict_proba(X_img)

# Reconstruir imagen segmentada
segmented = labels_img.reshape(img_size, img_size)

# Imagen con colores promedio de cada segmento
img_reconstructed = np.zeros_like(img)
for k in range(k_optimo_img):
    mask = labels_img == k
    color_mean = X_img[mask].mean(axis=0)
    img_reconstructed.reshape(-1, 3)[mask] = color_mean

# Visualización
fig, axes = plt.subplots(1, 4, figsize=(18, 4))

# Original
ax = axes[0]
ax.imshow(img)
ax.set_title('Imagen original')
ax.axis('off')

# Segmentación (etiquetas)
ax = axes[1]
ax.imshow(segmented, cmap='tab10')
ax.set_title(f'Segmentación GMM (K={k_optimo_img})')
ax.axis('off')

# Imagen reconstruida
ax = axes[2]
ax.imshow(img_reconstructed)
ax.set_title('Imagen reconstruida\n(colores promedio)')
ax.axis('off')

# Incertidumbre
ax = axes[3]
incertidumbre_img = entropy(probs_img) / np.log(k_optimo_img)
ax.imshow(incertidumbre_img.reshape(img_size, img_size), cmap='Reds')
ax.set_title('Mapa de incertidumbre')
ax.axis('off')

plt.suptitle('Segmentación de imagen con GMM', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"\nColores medios de cada segmento (RGB):")
for k in range(k_optimo_img):
    mask = labels_img == k
    color_mean = X_img[mask].mean(axis=0)
    print(f"  Segmento {k}: [{color_mean[0]:.2f}, {color_mean[1]:.2f}, {color_mean[2]:.2f}]")

## 7. Resumen y Conclusiones

### Conceptos Clave

1. **Gaussian Mixture Models (GMM)**
   - Modelo probabilístico que generaliza K-Means
   - Permite clusters elípticos con diferentes orientaciones
   - Soft clustering: proporciona probabilidades de pertenencia
   - Selección de K mediante BIC/AIC

2. **Clustering Espectral**
   - Transforma datos usando eigenvectores del laplaciano del grafo
   - Excelente para estructuras no convexas (anillos, espirales)
   - Parámetros críticos: tipo de afinidad (RBF, k-NN), gamma/n_neighbors

3. **Mean-Shift**
   - Encuentra modos de la densidad de datos
   - No requiere especificar K
   - Parámetro crítico: bandwidth

4. **Affinity Propagation**
   - Selecciona ejemplares (puntos reales) como representantes
   - No requiere K pero sensible a preference

5. **BIRCH**
   - Optimizado para grandes volúmenes de datos
   - Complejidad lineal O(n)
   - Ideal cuando los datos no caben en memoria

### Guía de Selección

| Situación | Algoritmo Recomendado |
|-----------|-----------------------|
| Clusters elípticos | GMM (covariance_type='full') |
| Soft clustering | GMM |
| Estructuras no convexas | Espectral |
| K desconocido | Mean-Shift, Affinity Propagation |
| Dataset muy grande | BIRCH |
| Ejemplares reales necesarios | Affinity Propagation |

---

## Referencias

- Bishop, C. M. (2006). Pattern Recognition and Machine Learning. Springer. (Capítulo 9: Mixture Models and EM)
- von Luxburg, U. (2007). A tutorial on spectral clustering. Statistics and Computing, 17(4), 395-416.
- Comaniciu, D., & Meer, P. (2002). Mean shift: A robust approach toward feature space analysis. IEEE TPAMI.
- Frey, B. J., & Dueck, D. (2007). Clustering by passing messages between data points. Science, 315(5814), 972-976.
- Zhang, T., Ramakrishnan, R., & Livny, M. (1996). BIRCH: an efficient data clustering method for very large databases. ACM SIGMOD.
- Scikit-learn documentation: https://scikit-learn.org/stable/modules/clustering.html

---


# EOF (End Of File)